# Notebook: Filter Sentences from Reviews to create Dataset for Annotation

## Packages

In [17]:
from bs4 import BeautifulSoup
from langdetect import detect
import pandas as pd
import json
import nltk
import uuid
from nltk.tokenize import sent_tokenize
import re

## Settings

In [18]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nils_hellwig/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Constants

In [19]:
REVIEWS_PATH = "reviews_dataset/reviews.csv"
RANDOM_STATE = 43

## Code

### Load Dataset

In [20]:
df_reviews = pd.read_csv(REVIEWS_PATH)

In [21]:
df_reviews

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization
0,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland","Die <LOC> schmeckte leicht säuerlich, der klei...",2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...
1,1,907168709,2005104,0,Wir sind hier nicht in Bayern!,2023-08-01,173stefanh,"München, Deutschland","Wir sind hier nicht in <LOC>, sind die freundl...",1.0,berlin,Augustiner am Gendarmenmarkt,de,"Wir sind hier nicht in Bayern, sind die freund..."
2,2,902540630,2005104,0,Heute leider zum abgewöhnen,2023-07-12,Patrick L,"Rüdersdorf, Deutschland",Ich freute mich sehr auf das Wienerschnitzel u...,3.0,berlin,Augustiner am Gendarmenmarkt,de,Ich freute mich sehr auf das Wienerschnitzel u...
3,3,895783388,2005104,0,Teils-Teils,2023-06-18,J9324NNpeterr,"Stellenbosch, Südafrika",Meine Erfahrungen in diesem Jahr waren gemisch...,4.0,berlin,Augustiner am Gendarmenmarkt,de,Meine Erfahrungen in diesem Jahr waren gemisch...
4,4,890917149,2005104,0,schlecht verwaltetes Chaos,2023-05-18,Harald F,"Berlin, Deutschland",ich hätte gerne etwas zum Essen gesagt. Aber e...,1.0,berlin,Augustiner am Gendarmenmarkt,de,ich hätte gerne etwas zum Essen gesagt. Aber e...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3596,3596,889964622,5207419,0,Nicht besonderes,2023-05-11,Mobile28959210843,"Mönchengladbach, Deutschland",Das Essen ist okay mehr aber auch nicht. Ambie...,3.0,mönchengladbach,Brauerei zum Stefanus,de,Das Essen ist okay mehr aber auch nicht. Ambie...
3597,3597,870345760,5207419,0,"Ungebührender Empfang, sehr gutes Essen",2022-12-01,Korich,"Görlitz, Deutschland",Zunächst einmal wurden wir von einem Mann mit ...,4.0,mönchengladbach,Brauerei zum Stefanus,de,Zunächst einmal wurden wir von einem Mann mit ...
3598,3598,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Wir sind nicht zum ersten Mal aber auch defini...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...
3599,3599,863841135,5207419,0,Gutes Essen mit Biergartenambiente,2022-10-09,788heinzn,"Köln, Deutschland","Gutes Essen, Portionen sehr groß (für uns zu g...",4.0,mönchengladbach,Brauerei zum Stefanus,de,"Gutes Essen, Portionen sehr groß (für uns zu g..."


### Filter Sentences

In [22]:
df_reviews_sentences = pd.DataFrame(columns=list(df_reviews.columns) + ['sentence_idx', "id"])

for idx, row in df_reviews.iterrows():
    sentences = sent_tokenize(row['text'], language="german")
    sentence_index = 0
    for sentence in sentences:
        if len(sentence) > 1:
            new_row = row.copy() 
            new_row['text'] = sentence 
            new_row['id'] = str(uuid.uuid4())
            new_row['sentence_idx'] = sentence_index
            df_reviews_sentences.loc[len(df_reviews_sentences)] = new_row
            sentence_index += 1

In [23]:
df_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id
0,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland","Die <LOC> schmeckte leicht säuerlich, der klei...",2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,0,dc3ff9e4-d354-468f-85de-1f7b6e944395
1,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland",Der Schweinebraten war lauwarm und der Kaisers...,2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,1,82674cf3-2a26-4f55-b8e4-c48c9c6c9727
2,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland",Ich bin so enttäuschst.,2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,2,f6974ed4-4d7c-490b-bb40-97be6a007ba1
3,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland",Gehe ich nicht mehr hin.,2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,3,147ca3a4-6ef0-49ca-89c1-4a5f57c9a7e3
4,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland","Schlecht kann ich auch woanders, besonders bei...",2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,4,65e6829a-0262-435d-8e07-b450fd950675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15344,3600,861397747,5207419,0,Das Schnitzelparadies!,2022-09-22,vereny2020,NaN,Das Essen schmeckt aber generell grandios.,5.0,mönchengladbach,Brauerei zum Stefanus,de,Ich bin im Schnitzelparadies <3 Die Schnitzel ...,1,a2bdc021-8d6f-4bc3-a15b-c9cd1d185424
15345,3600,861397747,5207419,0,Das Schnitzelparadies!,2022-09-22,vereny2020,NaN,Der Beilagensalat ist auch der Kracher: frisch...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Ich bin im Schnitzelparadies <3 Die Schnitzel ...,2,d23618cf-bb02-4fee-ba9f-f62d8de0411a
15346,3600,861397747,5207419,0,Das Schnitzelparadies!,2022-09-22,vereny2020,NaN,"Im Stefanus gibt es auch selbstgebrautes, sehr...",5.0,mönchengladbach,Brauerei zum Stefanus,de,Ich bin im Schnitzelparadies <3 Die Schnitzel ...,3,78656846-c442-4b62-916d-d16770c646c3
15347,3600,861397747,5207419,0,Das Schnitzelparadies!,2022-09-22,vereny2020,NaN,"Der Service ist sehr nett, zuvorkommend und he...",5.0,mönchengladbach,Brauerei zum Stefanus,de,Ich bin im Schnitzelparadies <3 Die Schnitzel ...,4,8afaf9d8-1d2d-4968-b026-11f1ec2e852c


### Filter Non-German Sentences

Even if the language code = "de", we have observed that some sentences are in other languagesthan german. These are excluded as well with the help of Google's language-detection.

In [24]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'Unknown'

df_reviews_sentences['detected_language'] = df_reviews_sentences['text'].apply(detect_language)
df_reviews_sentences[df_reviews_sentences['detected_language'] != 'de']

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
17,4,890917149,2005104,0,schlecht verwaltetes Chaos,2023-05-18,Harald F,"Berlin, Deutschland",Es war viel los.,1.0,berlin,Augustiner am Gendarmenmarkt,de,ich hätte gerne etwas zum Essen gesagt. Aber e...,3,3ba647eb-772d-45a8-9898-b4b658dc53c1,es
66,13,866452621,2005104,1,Mind this touri-gap,2022-10-29,452stephand,"Nürnberg, Deutschland",Fazit: mind this touri-gap,1.0,berlin,Augustiner am Gendarmenmarkt,de,Wunderschöne Location mit mangelhafter Leitung...,1,8b59cd4e-1d86-43db-bd1e-d63e6eca8d6e,en
93,17,856900319,2005104,1,Deftig bayrisch,2022-08-28,tatrab,"Kaiserslautern, Deutschland",Kann man mal hin,3.0,berlin,Augustiner am Gendarmenmarkt,de,Vorzubereitete Speisen in bayrischer Manier zu...,2,8a6c556b-eafa-49d2-a555-1b31b1ea8577,tl
113,22,847064431,2005104,2,Enttäuschend,2022-07-09,HerSchnu,"Erfurt, Deutschland",Fing schon gut an….,2.0,berlin,Augustiner am Gendarmenmarkt,de,In jedem bayerischem Gasthof isst man besser… ...,2,1ac90691-ad5a-4c8a-87bc-078663ec5005,en
131,25,883653345,947082,0,Tolle Burger!,2023-03-26,dittelbrunn,"Lübeck, Deutschland",Ketchup.,5.0,berlin,Burgermeister,de,Wir waren in der Filiale am S-Bahnhof Eberswal...,2,b9e2e8fc-6922-4ff7-91e2-6016112334b5,id
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15202,3564,866578862,13144674,6,Leckeres Essen in Mönchengladbach,2022-10-30,marcelbF2818DO,NaN,BRAVO!,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Ein tolles Restaurant! Ambiente und Empfang su...,3,7ae953d0-2257-4711-9082-b36ede81ecf4,pt
15231,3570,862649725,13144674,6,Einfach eine Freude und ein Genuss,2022-09-30,Bernd W,NaN,Klasse.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,"Okay, ich koche gern selbst. Wenn ich aber mal...",4,aa8187ef-604d-41ce-b7af-a67b8fedb91e,et
15241,3570,862649725,13144674,6,Einfach eine Freude und ein Genuss,2022-09-30,Bernd W,NaN,Place to be.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,"Okay, ich koche gern selbst. Wenn ich aber mal...",14,f336f882-8b1e-45fc-b7f4-0802bca43eec,en
15249,3572,861638375,13144674,6,"Handwerklich einwandfrei, Hohe Qualität und ei...",2022-09-24,Tilmann S,NaN,Top.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Eher zufällig kehrten wir als Ortsunkundige im...,4,db6f61b6-f4ec-4fca-86c0-debbe5bd2d2a,nl


In [25]:
df_reviews_sentences = df_reviews_sentences[df_reviews_sentences['detected_language'] == 'de']

### Balancing

In [26]:
df_balanced_reviews_sentences = df_reviews_sentences.groupby(['rating'], group_keys=False).apply(lambda x: x.sample(700, replace=False, random_state=RANDOM_STATE))
df_balanced_reviews_sentences.reset_index(drop=True, inplace=True)

### Randomisierung

In [27]:
df_balanced_reviews_sentences = df_balanced_reviews_sentences.sample(frac=1, random_state=RANDOM_STATE)  # 'frac=1' mischt den gesamten DataFrame
df_balanced_reviews_sentences.reset_index(drop=True, inplace=True)

### Check If Sentences are Unique

In [28]:
df_balanced_reviews_sentences.groupby(["review_id", "sentence_idx"]).size().reset_index(name="count")

,review_id,sentence_idx,count
0,845575616,0,1
1,845636774,0,1
2,845636774,1,1
3,845636774,2,1
4,845636774,3,1
...,...,...,...
3495,916485162,2,1
3496,916494891,1,1
3497,916494891,3,1
3498,916494891,5,1


### Store as .csv 

In [29]:
df_reviews_sentences.to_csv("reviews_dataset/reviews_sentences.csv")

In [30]:
df_balanced_reviews_sentences.to_csv("reviews_dataset/balanced_reviews_sentences.csv")

In [31]:
df_balanced_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
0,1906,875871390,8009090,16,Hausgemachte frische Pasta mit viel Auswahl un...,2023-01-21,Culture08862634821,"Dresden, Deutschland",Damit war die Sauce in Summe ein wenig zu salz...,4.0,dresden,Mamma Mia,de,Ein solides italienisches Restaurant in der Dr...,11,8f5a5889-8cce-40c4-a9d4-ec68c9ad3447,de
1,2443,911273338,1342313,1,Nicht nur überaus köstlich in romantischer Atm...,2023-08-18,Michael E,NaN,"Überragender Service, köstliche Speisen!Wir ko...",5.0,bielefeld,Jivino Enoteca,de,"Ungefragt wurde uns vorab serviert, was wir mö...",1,dfa4a582-b715-4fe5-ba4a-f143441b97bb,de
2,2192,877994419,1390048,1,Nicht für nur 2 Personen geeignet,2023-02-09,N_G_4000,"Fürth, Deutschland",Freundlichkeit und Qualität.Aber wenn Sie als ...,2.0,nürnberg,Bratwursthäusle,de,Im Garten bisher alles ok. Freundlichkeit und ...,1,57313614-0f5e-4855-80eb-f1fa735cdbb5,de
3,723,859195793,1190751,12,So lecker wie immer,2022-09-09,annaliebrern,NaN,Sie kochen für unsere Geburtstagsfeiern seit J...,5.0,frankfurt,Trattoria i Siciliani,de,Sie kochen für unsere Geburtstagsfeiern seit J...,0,80559f0d-8e77-4a6a-9970-5515bde2f0b8,de
4,563,850412704,718270,1,Lecker Kölsch,2022-07-28,kaiserdirk,"Leverkusen, Deutschland",Wir mögen Peters Kölsch.2.,4.0,köln,Peters Brauhaus,de,"Hier kehren wir immer wieder gerne ein, aus dr...",1,6089488a-38da-423e-8000-d8750df0dcf5,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,3113,887337929,7236591,0,Sehr empfehlenswert,2023-04-23,Thomas R,"Baden-Württemberg, Deutschland",Man wurde freundlich bedient.,4.0,karlsruhe,Restaurant Terra Mare,de,Das Essen war einfach herrlich und eine riesig...,1,13aa2cca-5eef-4ece-a2f3-c6449950985e,de
3496,3592,893275773,5207419,0,So gefällt Geburtstag feiern außer Haus,2023-06-03,Iris S,NaN,"Zum Servicepersonal, der Küche und dem „Chef i...",4.0,mönchengladbach,Brauerei zum Stefanus,de,Dem Anlass angemessen (Geburtstag) waren wir P...,6,0a0676a9-d275-4258-9398-12fb206cb5d7,de
3497,550,888907029,718270,0,Lecker Bier und tolle Atmosphäre,2023-05-04,Axel S,"Braunschweig, Deutschland",Atmosphäre ist toll und man sollte in jedem Fa...,4.0,köln,Peters Brauhaus,de,Wir waren hier zum Abendessen. Atmosphäre ist ...,1,5a1f3ec9-8e3d-4b73-b341-856183ebe793,de
3498,3175,889498235,7322855,4,Ein Reines Gedicht,2023-05-08,Tim R,NaN,Es schmeckt als ob die Kuh jeden Tag vom Bauer...,5.0,augsburg,SteakManufaktur,de,Zur Ambiente:Wir würden herzlich empfangen vom...,11,bf45e9b9-2317-4306-92f1-67d874158838,de
